In [1]:
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2
import numpy as np
#from matplotlib import pyplot as plt

import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')

In [2]:
configs = config_util.get_configs_from_pipeline_file(
    r"C:\Users\Philip Vangelov\PycharmProjects\pythonProject\TensorFlow\workspace\training_demo\models\faster_rcnn_resnet_v2_update_2\pipeline.config")
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(
    r"C:\Users\Philip Vangelov\PycharmProjects\pythonProject\TensorFlow\workspace\training_demo\models\faster_rcnn_resnet_v2_update_2\ckpt-16").expect_partial()


In [3]:
@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

In [13]:
category_index = label_map_util.create_category_index_from_labelmap(
    r"C:\Users\Philip Vangelov\PycharmProjects\pythonProject\TensorFlow\workspace\annotations\label_map.pbtxt")

IMAGE_PATH = r"C:\Users\Philip Vangelov\PycharmProjects\pythonProject\TensorFlow\workspace\training_demo\images\images_4\test\Brown Knapweed_B4A7881.jpg"

img = cv2.imread(IMAGE_PATH)
image_np = np.array(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
              for key, value in detections.items()}
detections['num_detections'] = num_detections

# detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
    image_np_with_detections,
    detections['detection_boxes'],
    detections['detection_classes'] + label_id_offset,
    detections['detection_scores'],
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=150,
    min_score_thresh=0.01,
    agnostic_mode=False,skip_labels=True)

array([[[ 35,  59,  51],
        [ 29,  53,  45],
        [ 31,  58,  49],
        ...,
        [ 27,  52,  42],
        [ 22,  48,  35],
        [ 35,  61,  48]],

       [[ 42,  65,  57],
        [ 33,  57,  49],
        [ 31,  55,  47],
        ...,
        [ 18,  43,  33],
        [ 38,  64,  51],
        [ 35,  63,  50]],

       [[ 43,  64,  56],
        [ 36,  59,  51],
        [ 30,  53,  45],
        ...,
        [ 30,  57,  47],
        [ 33,  61,  48],
        [ 50,  80,  67]],

       ...,

       [[ 78, 108,  89],
        [ 63,  91,  72],
        [ 39,  60,  45],
        ...,
        [128, 118, 111],
        [130, 121, 112],
        [124, 115, 106]],

       [[ 59,  90,  69],
        [ 67,  95,  76],
        [ 57,  78,  63],
        ...,
        [129, 119, 112],
        [127, 118, 109],
        [133, 124, 115]],

       [[ 58,  89,  68],
        [ 53,  81,  61],
        [ 43,  64,  49],
        ...,
        [135, 125, 118],
        [128, 119, 110],
        [141, 132, 123]]

In [14]:
plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
plt.show()

In [6]:
min_score = 0.1 # Chose min score for the detections
chosen_detections = []
for index,value in enumerate(zip(detections['detection_scores'],detections['detection_classes'])):
    if value[0] > min_score:
        chosen_detections.append(value)


In [7]:
print(chosen_detections)

[(0.9925029, 3), (0.9853914, 3), (0.97365665, 3), (0.9394185, 3), (0.9120266, 3), (0.8940744, 3), (0.8849586, 3), (0.83260113, 3), (0.82629913, 3), (0.8158715, 3), (0.79566854, 3), (0.7234796, 3), (0.70686424, 3), (0.69063663, 3), (0.68297297, 3), (0.6685154, 3), (0.6483147, 3), (0.5977895, 3), (0.5820531, 3), (0.518069, 3), (0.5051655, 3), (0.32462487, 3), (0.29748, 3), (0.23062256, 3), (0.20569548, 1), (0.17436408, 3), (0.17033729, 3), (0.14830528, 3)]
